In [30]:
from bs4 import BeautifulSoup
import requests
import lxml
import os
import csv
import pandas as pd
import plotly.express as px
from nltk.stem.snowball import SnowballStemmer
import nltk
nltk.download('stopwords')
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import *
import string
from nltk.corpus import stopwords
import re
from collections import Counter
from functools import reduce
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import math
import heapq as hq

In [7]:
url2='https://www.atlasobscura.com/places?page='
sort='&sort=likes_count'

### 1.1. Get the list of places And 1.2. Crawl places

We create the url txt file and html folders

In [10]:
def crawl():
    pathAncestor = os.path.join("./", "htmlpages") # Path
    #os.mkdir(pathAncestor) # create the folder in the path
    
    f = open("urlpages.txt", "w") #create txt file with urls
    count=1 #counting html files
    headpart='https://www.atlasobscura.com'
    
    for i in range(1, 401):
        os.makedirs(os.path.join(pathAncestor, 'page ' + str(i))) #create folder of html files
        
        url = url2 +str(i) +sort
        page_1 = requests.get(url)
        soup = BeautifulSoup(page_1.content, features='lxml')
        
        for a in soup.find_all('a', class_="content-card content-card-place"): #the class where the url is
            save=a.get('href')
            f.write(save +'\n')
            
            subdirectory = pathAncestor + "/page " + str(i) # select the corresponding folder to insert the html article
            article_name = "/article_"+str(count)+".html" 
            count+=1
            complete_path = subdirectory + article_name # insert the new complete path where create the html file
            
            with open(complete_path, "wb") as htmlFile:
                link = headpart + save
                try:
                    page_2 = requests.get(link) # request the page
                except:
                    with open("failureRequest.txt", "a") as error: # if we loose the request place, we put into a file the link doesn't download well, then we set the "urlpages.txt" with these link
                        error.write(link)
                        error.close()

                soup = BeautifulSoup(page_2.text, features='lxml')

                htmlFile.write(soup.encode('utf-8'))
                htmlFile.close()

    f.close()

In [11]:
crawl()

### 1.3 Parse downloaded pages


We create the tsv files for each html file

In [2]:
def scrap(tsv_writer, article): 
    
    with open(article, 'r', encoding="utf-8") as out_file: # for each html article downloaded scrape it!
        contents = out_file.read()
        soup = BeautifulSoup(contents, features="lxml") #parse the text
        
    try:
        roi_header = soup.find_all("div", {"class": "DDPage__header-container grid-row"})[0]
        placeName = roi_header.find("h1", {"class":"DDPage__header-title"}).text.strip()

    except:
        placeName = " "
    try:
        related_list = soup.find("div",{
            "class":"card-grid CardRecircSection__card-grid js-inject-gtm-data-in-child-links",
            "data-gtm-template":"DDP Footer Recirc Lists"
        })
        
        placeRelatedLists = [data.find("span").text.strip() for data in related_list.find_all("h3")]
    except:
        placeRelatedLists=''
        
    try:
        related_places = soup.find("div",{
            "class":"card-grid CardRecircSection__card-grid js-inject-gtm-data-in-child-links",
            "data-gtm-template":"DDP Footer Recirc Related"
        })
        placeRelatedPlaces = [data.find("span").text.strip() for  data in related_places.find_all("h3")]
    except:
        placeRelatedPlaces=''
    try:
        placeDesc = soup.find_all("div", {"class": "DDP__body-copy"})[0]
        placeDesc = "".join([p.text.strip() for p in placeDesc.find_all("p")])
    except:
        placeDesc=''
    try:
        location = roi_header.find("div", {"class":"DDPage__header-place-location"}).text.strip()
    except:
        location=''
    try:
        counters = roi_header.find_all("div",{"class":"title-md item-action-count"})
        numPeopleVisited=int(counters[0].text.strip())
    except:
        numPeopleVisited=''
    try:
        numPeopleWant = int(counters[1].text.strip())
    except:
        numPeopleWant=''
    try:
        placeShortDesc = roi_header.find("h3", {"class":"DDPage__header-dek"}).text.strip()
    except:
        placeShortDesc=''
    try:
        sidebar = soup.find("div", {"class":"DDPageSiderail"})
        nearby_places = sidebar.find_all("div",{"class":"DDPageSiderailRecirc__item-title"})
        positions =  sidebar.find("div",{"class":"DDPageSiderail__coordinates js-copy-coordinates"}).attrs["data-coordinates"].split(",")
        placeAlt= float(positions[0].strip())
    except:
        placeAlt=''
    try:
        placeLong = float(positions[1].strip())
    except:
        placeLong=''
    try:
        placeNearby = [place.text for place in nearby_places]
    except:
        placeNearby=''
    try:
        placeAddress = [x.text for x in soup.find_all('div', {'class': 'DDPage__header-place-location'})][0]
        
    except:
        placeAddress=''
        
    try:
        footer = soup.find("div", {"id":"ugc-module"})
        Editors = footer.find_all("a",{"class":"DDPContributorsList__contributor"})
        placePubDate = footer.find("div",{"class":"DDPContributor__name"}).text
        placePubDate = datetime.strptime(placePubDate,"%B %d, %Y")
    except:
        placePubDate=''
    try:
        #10-placeEditors, 11-placePubDate, 2-placeTags
        placeEditors = [editors.find("span").text if editors.find("span") else editors.text for editors in Editors]
    except:
        placeEditors=''
    try:
        placeTags = [item.get_text("").replace("\n","") for item in soup.find_all("a",{"class":"itemTags__link js-item-tags-link"})]
        # set_trace(header="sidebar")
    except:   
        placeTags=''
    placeURL=''

    tsv_writer.writerow([placeName, placeTags, numPeopleVisited, numPeopleWant, 
                                placeDesc, placeShortDesc, placeNearby, placeAddress, placeAlt,
                                placeLong, placeEditors, placePubDate,placeRelatedLists,placeRelatedPlaces,placeURL])


In [12]:
def create_tsv(input_path):
    path = str("./"+input_path)

    filenames = os.listdir(path)
    for i in range(1, 401):
        filenames = os.listdir(path + '/page ' + str(i))

        for file in filenames:
            with open(path + '/page ' + str(i) + '/article_'+str(file.split("_")[1].replace(".html", ""))+'.tsv', 'w', encoding="utf-8", newline='') as out_file: # create for each html article its article_i.tsv 
                tsv_writer = csv.writer(out_file, delimiter='\t')
                tsv_writer.writerow(['placeName', 'placeTags', 'numPeopleVisited', 'numPeopleWant', 
                                'placeDesc', 'placeShortDesc', 'placeNearby', 'placeAddress', 'placeAlt',
                                'placeLong', 'placeEditors', 'placePubDate','placeRelatedLists','placeRelatedPlaces','placeURL'])
                scrap(tsv_writer, path + '/page ' + str(i) + "/" + file)

In [13]:
create_tsv('htmlpages')

We create the tsv file 

In [17]:
def dataFr():
    path = "./htmlpages"
    data = pd.DataFrame()

    for num in range(1, 401):
        filenames = os.listdir(path + '/page ' + str(num))
        for file in filenames:
            if file.endswith(".tsv"): # look for tsv files
                with open(path + '/page ' + str(num) + '/article_'+str(file.split("_")[1]), 'r', encoding="utf-8", newline='') as file_tsv:
                        df = pd.read_csv(file_tsv,sep = "\t")
                        data = pd.concat([data,df])
                            
    data.to_csv('/home/sophja/Documenti/ADM/HW3/data.tsv', sep='\t', index=False)


In [18]:
dataFr() #we call the tsv function to crete our dataset

I forgot the url field

In [2]:
data=pd.read_csv('/home/sophja/Documenti/ADM/data.tsv', sep='\t')

In [141]:
with open ('/home/sophja/Documenti/ADM/HW3/urlpages2.txt', 'r') as url_file:
    for url in url_file:
        url2=url.replace('\n','').replace('/',' ').split(' ')[2].replace('-',' ')
        data.loc[data['placeName'].str.lower() == url2, 'placeURL'] = url.replace('\n','')
        

In [31]:
data.to_csv('/home/sophja/Documenti/ADM/data2.tsv', sep='\t', index=False)

In [171]:
data.head()

,placeName,placeTags,numPeopleVisited,numPeopleWant,placeDesc,placeShortDesc,placeNearby,placeAddress,placeAlt,placeLong,placeEditors,placePubDate,placeRelatedLists,placeRelatedPlaces,placeURL
0,Catacombes de Paris,"['ossuaries', 'memento mori', 'catacombs and c...",4457.0,7059.0,"In 2004, Parisian police were assigned to do a...","The vast, legendary catacombs hold secrets muc...","['Sculptures de Décure', 'Arago Medallions', ""...","Paris, France",48.8343,2.3322,"['CPilgrim', 'ack sed', 'ramonrodz2212', 'ging...",NaN,"['19 Catacombs Sure to Tingle Your Spine', ""Th...","['Ossario di San Martino', 'Leuk Charnel House...",/places/catacombes-de-paris
1,Dream House,"['optical oddities', 'eccentric homes', 'music...",766.0,6007.0,"When walking down Church Street in Tribeca, ke...","La Monte Young and Marian Zazeela's ""Dream Hou...","['Aretha Franklin Subway Tributes', 'Farm.One'...","Manhattan, New York",40.7185,-74.0048,"['seanmattison', 'erosika', 'mbison', 'Gray', ...",NaN,['30 Unexpected Places to Have a Joyful Advent...,"['Callejon de Hamel', 'Casa Neverlandia', 'Cas...",/places/dream-house
2,City Hall Station,"['subways', 'subterranean', 'infrastructure', ...",1834.0,8608.0,The first New York City subway was built and o...,A beautiful and abandoned New York subway stat...,"['African Burial Ground National Monument', 'T...","Manhattan, New York",40.7134,-74.0046,"['Rebekah Otto', 'charding407', 'mbison', 'Ann...",NaN,['30 Unexpected Places to Have a Joyful Advent...,"['Crystal Palace Subway', 'Moscow Metro Statio...",/places/city-hall-station
3,The Evolution Store,"['wonder cabinets', 'purveyors of curiosities'...",2199.0,6953.0,Evolution stands out among the clothing stores...,A terrific purveyor of natural history objects...,"[""Merchant's House Museum"", 'The Brown Buildin...","Manhattan, New York",40.7281,-73.9948,"['mbison', 'ramosju', 'Lauren Levesque', 'Unus...",NaN,"[""The World's Top 100 Wonders in 2018"", ""New Y...","['Kunstkammer Georg Laue', 'Ospedale delle Bam...",NaN
4,The Winchester Mystery House,"['follies and grottoes', 'outsider architectur...",3724.0,5208.0,In 1886 an eccentric woman named Sarah Winches...,A peculiar mansion built by the troubled heir ...,"['Santana Row Chess Plaza', 'Rosicrucian Park'...","San Jose, California",37.3189,-121.9506,"['mbison', 'Avoiding Regret', 'Collin', 'marle...",NaN,['12 Places to See the Work of Women Architect...,"['Casa de Piedra (Stone House)', 'Portmeirion ...",NaN


In [3]:
data.info()
data=data.dropna(subset=['placeDesc'])



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7198 entries, 0 to 7197
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   placeName           7198 non-null   object 
 1   placeTags           7198 non-null   object 
 2   numPeopleVisited    7193 non-null   float64
 3   numPeopleWant       7193 non-null   float64
 4   placeDesc           7193 non-null   object 
 5   placeShortDesc      7193 non-null   object 
 6   placeNearby         7193 non-null   object 
 7   placeAddress        7192 non-null   object 
 8   placeAlt            7193 non-null   float64
 9   placeLong           7193 non-null   float64
 10  placeEditors        7193 non-null   object 
 11  placePubDate        0 non-null      float64
 12  placeRelatedLists   2759 non-null   object 
 13  placeRelatedPlaces  7159 non-null   object 
 14  placeURL            4123 non-null   object 
dtypes: float64(5), object(10)
memory usage: 843.6+ KB


### 2.1. Conjunctive query

clean the description field

In [4]:
def cleanWords(field):
    stemmer = PorterStemmer()

    snowstem = SnowballStemmer('english')
    new_w=[snowstem.stem(word) for word in field.split(' ')] #stem the words
    new_w = list(map(lambda x: x.replace(x, x.translate(str.maketrans('', '', string.punctuation))), new_w)) #remove punctuation  
    stop_words = set(stopwords.words('english'))
    new_w= list(filter(lambda word: word not in stop_words, new_w)) #remove stop words
    new_w=list(map(lambda x: re.sub('[\W_]+', '', x),new_w)) #remove everything is not letters or number 
    
    return new_w

In [5]:
data['placeDesc_new'] = data.placeDesc.apply(lambda x: cleanWords(x))

In [186]:
data.head(10)

,placeName,placeTags,numPeopleVisited,numPeopleWant,placeDesc,placeShortDesc,placeNearby,placeAddress,placeAlt,placeLong,placeEditors,placePubDate,placeRelatedLists,placeRelatedPlaces,placeURL,placeDesc_new
0,Catacombes de Paris,"['ossuaries', 'memento mori', 'catacombs and c...",4457.0,7059.0,"In 2004, Parisian police were assigned to do a...","The vast, legendary catacombs hold secrets muc...","['Sculptures de Décure', 'Arago Medallions', ""...","Paris, France",48.8343,2.3322,"['CPilgrim', 'ack sed', 'ramonrodz2212', 'ging...",NaN,"['19 Catacombs Sure to Tingle Your Spine', ""Th...","['Ossario di San Martino', 'Leuk Charnel House...",/places/catacombes-de-paris,"[2004, parisian, polic, assign, train, exercis..."
1,Dream House,"['optical oddities', 'eccentric homes', 'music...",766.0,6007.0,"When walking down Church Street in Tribeca, ke...","La Monte Young and Marian Zazeela's ""Dream Hou...","['Aretha Franklin Subway Tributes', 'Farm.One'...","Manhattan, New York",40.7185,-74.0048,"['seanmattison', 'erosika', 'mbison', 'Gray', ...",NaN,['30 Unexpected Places to Have a Joyful Advent...,"['Callejon de Hamel', 'Casa Neverlandia', 'Cas...",/places/dream-house,"[walk, church, street, tribeca, keep, eye, bla..."
2,City Hall Station,"['subways', 'subterranean', 'infrastructure', ...",1834.0,8608.0,The first New York City subway was built and o...,A beautiful and abandoned New York subway stat...,"['African Burial Ground National Monument', 'T...","Manhattan, New York",40.7134,-74.0046,"['Rebekah Otto', 'charding407', 'mbison', 'Ann...",NaN,['30 Unexpected Places to Have a Joyful Advent...,"['Crystal Palace Subway', 'Moscow Metro Statio...",/places/city-hall-station,"[first, new, york, citi, subway, built, oper, ..."
3,The Evolution Store,"['wonder cabinets', 'purveyors of curiosities'...",2199.0,6953.0,Evolution stands out among the clothing stores...,A terrific purveyor of natural history objects...,"[""Merchant's House Museum"", 'The Brown Buildin...","Manhattan, New York",40.7281,-73.9948,"['mbison', 'ramosju', 'Lauren Levesque', 'Unus...",NaN,"[""The World's Top 100 Wonders in 2018"", ""New Y...","['Kunstkammer Georg Laue', 'Ospedale delle Bam...",NaN,"[evolut, stand, among, cloth, store, restaur, ..."
4,The Winchester Mystery House,"['follies and grottoes', 'outsider architectur...",3724.0,5208.0,In 1886 an eccentric woman named Sarah Winches...,A peculiar mansion built by the troubled heir ...,"['Santana Row Chess Plaza', 'Rosicrucian Park'...","San Jose, California",37.3189,-121.9506,"['mbison', 'Avoiding Regret', 'Collin', 'marle...",NaN,['12 Places to See the Work of Women Architect...,"['Casa de Piedra (Stone House)', 'Portmeirion ...",NaN,"[1886, eccentr, woman, name, sarah, winchest, ..."
5,The Wave,"['martian landscapes', 'geological oddities', ...",453.0,5430.0,"190 million years ago, one of the greatest geo...",Rippling sand dunes frozen in the Arizona rock.,"['The Wahweap Hoodoos', 'Navajo Bridge', ""Lee'...","Marble Canyon, Arizona",36.9961,-112.0061,"['renakaba', 've897969', 'rscottjones', 'mateo...",NaN,"[""The World's Top 100 Wonders in 2018"", 'Favor...","['Kanaio Coast', 'Cabezon Peak', 'The Smoking ...",NaN,"[190, million, year, ago, one, greatest, geolo..."
6,Fingal's Cave,"['jules verne', 'geological oddities', 'caves'...",536.0,5713.0,"Queen Victoria, Matthew Barney, Jules Verne, a...",This astonishingly geometric cave has inspired...,"['Iona Abbey', 'Clach Na Criche (The Wishing S...","Isle of Staffa, Scotland",56.4314,-6.3413,"['Dylan', 'KenJ', 'Jaszmina Szendrey', 'Martin...",NaN,"[""The World's Top 100 Wonders in 2018"", 'Favor...","['Kong Lor Cave', 'Marble Caves of Chile Chico...",NaN,"[queen, victoria, matthew, barney, jule, verne..."
7,Highgate Cemetery,"['rivals week', 'vampires', 'hoaxes', 'occult'...",2626.0,8213.0,"Opened in 1839, Highgate is one of London’s mo...",London's creepiest cemetery was once the site ...,"[""World's Largest Potted Plant"", 'Dick Whittin...","London, England",51.5675,

We create the vocabulary

In [6]:
vocabolary=Counter(reduce(lambda x, y: x + y, data.placeDesc_new))

In [7]:
word_dict = {} #dictionary with word as key and id as value
item_id = 1
for word in vocabolary.keys():
    word_dict[word] = item_id
    item_id += 1

we create th inverted index

In [8]:
inverted_idx = {}
for word, item_id in word_dict.items():
    inverted_idx[item_id] = list(data[data.placeDesc_new.apply(lambda row: word in row)].index)

In [17]:
snowstem = SnowballStemmer('english')
def stem_text(txt):
    return [snowstem.stem(word) for word in txt.split(' ')]

In [18]:
def serch(query_str):
    data_conj=pd.DataFrame(columns=['placeName', 'placeDesc', 'placeURL'])
    results = [set(inverted_idx[item]) for item in [word_dict[word] for word in stem_text(query_str)]]
    conjuctive=set(reduce(lambda x,y: x.intersection(y), results))
    for doc in conjuctive:
        new_row = {'placeName': data.loc[doc]['placeName'], 'placeDesc': data.loc[doc]['placeDesc'], 'placeURL': data.loc[doc]['placeURL']}
        #append row to the dataframe
        
        data_conj = data_conj.append(new_row, ignore_index=True)
    data_conj=data_conj.dropna(subset=['placeName'])
    return data_conj
        


In [19]:
serch('america museum')

,placeName,placeDesc,placeURL
0,Whale Museum,"Nestled in the San Juan Islands, the Whale Mus...",/places/whale-museum
1,The Palace Light Bulb,The Centennial Bulb is the world’s longest bur...,NaN
2,Cryptozoology & Paranormal Museum,"The spookiest, most unexplainable phenomena al...",NaN
3,Cave Hill Cemetery,Holding the bodies of a number of influential ...,/places/cave-hill-cemetery
4,Salvador Dalí Museum,The world’s largest collection of art by Salva...,NaN
...,...,...,...
120,"Harriet Beecher Stowe, Slavery to Freedom Museum",This early brick Georgian townhouse sits incon...,NaN
121,The U.S. Pizza Museum,"When it comes to New York and Chicago, pizza i...",NaN
122,Fairbanks House,"The Fairbanks House has stood in Dedham, Massa...",/places/fairbanks-house
123,"Santa Claus, Indiana","Not only that, but you can can totally send a ...",NaN


### 2.2) Conjunctive query & Ranking score

2.2.1) Inverted index

In [24]:
#tf t,d = numero di parole /lunghezza campo
#idf t = log(N/df)  N is the total number of documents in the collection, and df is the number of documents a term occurs in. 
#w t,d = tf * idf The TF-IDF is the dot product of the TF and IDF, so calculated as:
inverted_idx2 = {}
term_idf = {}

for word, item_id in word_dict.items():
    result = []
    docList = inverted_idx[item_id]
    idf = math.log(len(data)/len(docList))
    for doc in docList:
        tf = (data.loc[doc].placeDesc.count(word))/len(data.loc[doc].placeDesc)
        tfidf = tf * idf
        result.append((doc, tfidf))
    term_idf[item_id] = idf
    inverted_idx2[item_id] = result

### 4. Visualizing the most relevant places

We use mapbox to visualize the places on a map, I know this works I tried in another dataset and not the result of the third exercise

In [33]:
def maps(data):  

    fig = px.scatter_mapbox(data_prova, lat="placeAlt", lon="placeLong", hover_name="placeName", hover_data=["numPeopleWant", "numPeopleVisited", "placeAddress"],
                        color_discrete_sequence=["fuchsia"], zoom=3, height=300)
    fig.update_layout(mapbox_style="open-street-map")
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()